In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import pandas_ta as ta
data = yf.download(tickers = '^RUI', start = '2012-03-11',end = '2022-07-10')
data.drop(['Volume'], axis=1, inplace=True)
data.head(10)

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close
Date,,,,,
2012-03-12,759.429993,760.619995,756.840027,759.130005,759.130005
2012-03-13,761.450012,773.030029,761.450012,772.979980,772.979980
2012-03-14,773.190002,774.679993,769.280029,771.630005,771.630005
2012-03-15,772.380005,776.469971,770.739990,776.419983,776.419983
2012-03-16,776.559998,778.099976,775.640015,777.130005,777.130005
2012-03-19,777.119995,782.549988,776.070007,780.090027,780.090027
2012-03-20,778.030029,778.299988,773.119995,777.369995,777.369995
2012-03-21,777.559998,778.650024,774.760010,776.150024,776.150024
2012-03-22,773.640015,773.640015,768.000000,770.270020,770.270020


In [67]:
data = data[data.Open != data.Close]
data = data.reset_index()
data

,Date,Open,High,Low,Close,Adj Close
0,2012-03-12,759.429993,760.619995,756.840027,759.130005,759.130005
1,2012-03-13,761.450012,773.030029,761.450012,772.979980,772.979980
2,2012-03-14,773.190002,774.679993,769.280029,771.630005,771.630005
3,2012-03-15,772.380005,776.469971,770.739990,776.419983,776.419983
4,2012-03-16,776.559998,778.099976,775.640015,777.130005,777.130005
...,...,...,...,...,...,...
2590,2022-07-01,2075.000000,2101.600098,2059.290039,2099.540039,2099.540039
2591,2022-07-05,2092.550049,2106.070068,2053.879883,2105.659912,2105.659912
2592,2022-07-06,2106.159912,2124.419922,2091.699951,2110.409912,2110.409912
2593,2022-07-07,2115.949951,2148.350098,2115.949951,2144.300049,2144.300049


In [68]:
data['Heiken_Close'] = (data.Open+data.Close+data.High+data.Low)/4
data['Heiken_Open'] = data['Open']
for i in range(1, len(data)):
    data['Heiken_Open'][i] = (data.Heiken_Open[i-1]+data.Heiken_Close[i-1])/2

data['Heiken_High'] = data[['High', 'Heiken_Open', 'Heiken_Close']].max(axis=1)
data['Heiken_Low'] = data[['Low', 'Heiken_Open', 'Heiken_Close']].min(axis=1)
data.dropna(inplace=True)
data.head(10)

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Open,High,Low,Close,Adj Close,Heiken_Close,Heiken_Open,Heiken_High,Heiken_Low
0,2012-03-12,759.429993,760.619995,756.840027,759.130005,759.130005,759.005005,759.429993,760.619995,756.840027
1,2012-03-13,761.450012,773.030029,761.450012,772.979980,772.979980,767.227509,759.217499,773.030029,759.217499
2,2012-03-14,773.190002,774.679993,769.280029,771.630005,771.630005,772.195007,763.222504,774.679993,763.222504
3,2012-03-15,772.380005,776.469971,770.739990,776.419983,776.419983,774.002487,767.708755,776.469971,767.708755
4,2012-03-16,776.559998,778.099976,775.640015,777.130005,777.130005,776.857498,770.855621,778.099976,770.855621
5,2012-03-19,777.119995,782.549988,776.070007,780.090027,780.090027,778.957504,773.856560,782.549988,773.856560
6,2012-03-20,778.030029,778.299988,773.119995,777.369995,777.369995,776.705002,776.407032,778.299988,773.119995
7,2012-03-21,777.559998,778.650024,774.760010,776.150024,776.150024,776.780014,776.556017,778.650024,774.760010
8,2012-03-22,773.640015,773.640015,768.000000,770.270020,770.270020,771.387512,776.668015,776.668015,768.000000
9,2012-03-23,770.650024,774.010010,766.869995,772.929993,772.929993,771.115005,774.027764,774.027764,766.869995


In [69]:
data["EMA10"] = ta.ema(data.Close, length=10)
data["EMA30"] = ta.ema(data.Close, length=30)
data['RSI'] = ta.rsi(data.Close, length=12)

In [70]:
import plotly.graph_objects as go
from datetime import datetime
dfpl = data[2500:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Heiken_Open'],
                high=dfpl['Heiken_High'],
                low=dfpl['Heiken_Low'],
                close=dfpl['Heiken_Close']),
                     go.Scatter(x=dfpl.index, y=dfpl.EMA10, line=dict(color='red', width=1), name="EMA20"),
                     go.Scatter(x=dfpl.index, y=dfpl.EMA30, line=dict(color='blue', width=1), name="EMA50")])

fig.show()

In [71]:
dfpl = data[2500:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()

In [72]:
import math
def HeikenSignal1(df):
    signal1=[0]*len(df)
    # signal1=[0 for i in rqnge(len(df))]
    for i in range(1, len(df)):
        #df.Heiken_High[i-1]<=df.Heiken_Open[i-1])and
        PreviousHeikenBody = math.fabs(df.Heiken_Open[i-1]-df.Heiken_Close[i-1])
        ratio = 1.5
        if ( 
            (df.Heiken_High[i-1]-max(df.Heiken_Open[i-1], df.Heiken_Close[i-1]))
            /PreviousHeikenBody>ratio and
            (min(df.Heiken_Open[i-1], df.Heiken_Close[i-1]) - df.Heiken_Low[i-1])
            /PreviousHeikenBody>ratio and
            (df.Heiken_Open[i]<df.Heiken_Close[i] and df.Heiken_Low[i]>=df.Heiken_Open[i]) ):
            signal1[i]=2
        #df.Heiken_High[i-1]<=df.Heiken_Open[i-1])and
        if ( (df.Heiken_High[i-1]-max(df.Heiken_Open[i-1], df.Heiken_Close[i-1]))
            /PreviousHeikenBody>ratio>ratio and
            (min(df.Heiken_Open[i-1], df.Heiken_Close[i-1]) - df.Heiken_Low[i-1])
            /PreviousHeikenBody>ratio>ratio and
            (df.Heiken_Open[i]>df.Heiken_Close[i] and df.Heiken_High[i]<=df.Heiken_Open[i]) ):
            signal1[i]=1
    df['HeikenSignal1'] = signal1
    
def totalSignal(df):
    ordersignal=[0]*len(df)
    for i in range(0, len(df)):
        if (df.EMA10[i]>df.EMA30[i] and df.Heiken_Open[i]<df.EMA10[i] 
            and df.Heiken_Close[i]>df.EMA10[i] 
            and df.HeikenSignal1[i] == 2):
            ordersignal[i]=2
        if (df.EMA10[i]<df.EMA30[i] and df.Heiken_Open[i]>df.EMA10[i] 
            and df.Heiken_Close[i]<df.EMA10[i]
            and df.HeikenSignal1[i] == 1):
            ordersignal[i]=1
    df['ordersignal']=ordersignal
HeikenSignal1(data)
totalSignal(data)

In [73]:
data.dropna(inplace=True)
data.reset_index(inplace=True)
data

,index,Date,Open,High,Low,Close,Adj Close,Heiken_Close,Heiken_Open,Heiken_High,Heiken_Low,EMA10,EMA30,RSI,HeikenSignal1,ordersignal
0,29,2012-04-23,759.880005,759.880005,751.340027,756.210022,756.210022,756.827515,763.984163,763.984163,751.340027,762.950352,770.468669,41.126537,0,0
1,30,2012-04-24,756.280029,760.799988,756.280029,758.690002,758.690002,758.012512,760.405839,760.799988,756.280029,762.175743,769.708755,43.685923,0,0
2,31,2012-04-25,763.219971,769.679993,763.219971,769.369995,769.369995,766.372482,759.209176,769.679993,759.209176,763.483789,769.686899,53.236589,0,0
3,32,2012-04-26,769.239990,775.849976,767.590027,774.659973,774.659973,771.834991,762.790829,775.849976,762.790829,765.515822,770.007743,57.162267,0,0
4,33,2012-04-27,776.130005,778.520020,773.070007,776.780029,776.780029,776.125015,767.312910,778.520020,767.312910,767.563860,770.444664,58.678839,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2561,2590,2022-07-01,2075.000000,2101.600098,2059.290039,2099.540039,2099.540039,2083.857544,2093.644662,2101.600098,2059.290039,2100.126052,2150.645263,45.189773,0,0
2562,2591,2022-07-05,2092.550049,2106.070068,2053.879883,2105.659912,2105.659912,2089.539978,2088.751103,2106.070068,2053.879883,2101.132209,2147.742982,46.179109,0,0
2563,2592,2022-07-06,2106.159912,2124.419922,2091.699951,2110.409912,2110.409912,2108.172424,2089.145540,2124.419922,2089.145540,2102.819064,2145.334397,46.989296,2,0
2564,2593,2022-07-07,2115.949951,2148.350098,2115.949951,2144.300049,2144.300049,2131.137512,2098.658982,2148.350098,2098.658982,2110.361061,2145.267665,52.548959,0,0


In [74]:
import numpy as np
def pointpos(x):
    if x['ordersignal']==1:
        return x['High']+1e-4
    elif x['ordersignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan
data['pointpos'] = df=data.apply(lambda row: pointpos(row), axis=1)

In [75]:
dfpl = data[2000:2100]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Heiken_Open'],
                high=dfpl['Heiken_High'],
                low=dfpl['Heiken_Low'],
                close=dfpl['Heiken_Close']),
                     go.Scatter(x=dfpl.index, y=dfpl.EMA10, line=dict(color='red', width=1), name="EMA10"),
                     go.Scatter(x=dfpl.index, y=dfpl.EMA30, line=dict(color='blue', width=1), name="EMA30")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [77]:
dfpl = data[2100:2200]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                     go.Scatter(x=dfpl.index, y=dfpl.EMA10, line=dict(color='red', width=1), name="EMA10"),
                     go.Scatter(x=dfpl.index, y=dfpl.EMA30, line=dict(color='blue', width=1), name="EMA30")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [78]:
# StopLoss from signal
SLSignal = [0] * len(df)
SLbackcandles = 1
for row in range(SLbackcandles, len(data)):
    mi=1e10
    ma=-1e10
    if data.ordersignal[row]==1:
        for i in range(row-SLbackcandles, row+1):
            ma = max(ma,data.High[i])
        SLSignal[row]=ma
    if data.ordersignal[row]==2:
        for i in range(row-SLbackcandles, row+1):
            mi = min(mi,data.Low[i])
        SLSignal[row]=mi
        
data['SLSignal']=SLSignal

In [79]:
dfpl = df[:]
def SIGNAL():
    return dfpl.ordersignal

In [88]:
dfpl = data[:]
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.2
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        TPSLRatio = 1
        
        #if len(self.trades)>0:            
        #    if self.trades[-1].is_long and self.data.RSI[-1]>=70:
        #        self.trades[-1].close()
        #    elif self.trades[-1].is_short and self.data.RSI[-1]<=30:
        #        self.trades[-1].close()
                   
        if len(self.trades)>0:            
            if self.trades[-1].is_long and self.data.Heiken_Open[-1]>=self.data.Heiken_Close[-1]:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.Heiken_Open[-1]<=self.data.Heiken_Close[-1]:
                self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:   
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.SLSignal[-1]
            tp1 = self.data.Close[-1]-(sl1 - self.data.Close[-1])*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2565.0
Duration                               2565.0
Exposure Time [%]                    6.313328
Equity Final [$]                  8866.468872
Equity Peak [$]                  22696.490112
Return [%]                         -11.335311
Buy & Hold Return [%]              183.213642
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -63.756114
Avg. Drawdown [%]                  -20.445979
Max. Drawdown Duration                 1487.0
Avg. Drawdown Duration                 294.25
# Trades                                 39.0
Win Rate [%]                        41.025641
Best Trade [%]                       4.907738
Worst Trade [%]                     -2.120535
Avg. Trade [%]                    

In [89]:
bt.plot(show_legend=False)

Row(id='4840', ...)

In [95]:
from backtesting import Strategy, Backtest
import numpy as np

class MyCandlesStrat(Strategy):
    sltr = 1
    mysize = 0.1
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        sltr = self.sltr
        for trade in self.trades: 
            if trade.is_long: 
                trade.sl = max(trade.sl or -np.inf, self.data.Close[-1] - self.sltr)
            else:
                trade.sl = min(trade.sl or np.inf, self.data.Close[-1] + self.sltr) 

        if self.signal1==2 and len(self.trades)==0:
            self.sltr = self.data.Close[-1] - self.data.SLSignal[-1]
            sl1 = self.data.Close[-1] - self.sltr
            self.buy(sl=sl1, size=self.mysize)
        elif self.signal1==1 and len(self.trades)==0:
            self.sltr = self.data.SLSignal[-1] - self.data.Close[-1]
            sl1 = self.data.Close[-1] + self.sltr
            self.sell(sl=sl1, size=self.mysize)

bt = Backtest(dfpl, MyCandlesStrat, cash=10000, margin=1/50, commission=.00)
stat = bt.run()
stat

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                    2565.0
Duration                               2565.0
Exposure Time [%]                   17.537023
Equity Final [$]                 11511.000305
Equity Peak [$]                   25492.60675
Return [%]                          15.110003
Buy & Hold Return [%]              183.213642
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -54.845731
Avg. Drawdown [%]                   -7.969386
Max. Drawdown Duration                 1179.0
Avg. Drawdown Duration             103.958333
# Trades                                 35.0
Win Rate [%]                        34.285714
Best Trade [%]                      10.055847
Worst Trade [%]                     -2.709465
Avg. Trade [%]                    

In [96]:
bt.plot(show_legend=False)

Row(id='6216', ...)